<a href="https://colab.research.google.com/github/jcowart1987/SalesPredictions/blob/main/DataPredictionsJC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Imports
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, \
OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer

from sklearn import set_config
set_config(display='diagram')

from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression

## Regression Metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error


In [ ]:
df = pd.read_csv('/content/sales_predictions (1).csv')

In [ ]:
df.duplicated().sum()

0

In [ ]:
df.replace(['LF', 'reg', 'low fat'], ['Low Fat', 'Regular', 'Low Fat'], inplace=True)

In [ ]:
Out_size = {'Small': 0, 'Medium': 1, 'High': 2}
df['Outlet_Size'] = df['Outlet_Size'].replace(Out_size)

In [ ]:
Loc_Type = {'Tier 1': 0, 'Tier 2': 1, 'Tier 3': 2}
df['Outlet_Location_Type'] = df['Outlet_Location_Type'].replace(Loc_Type)

df['Outlet_Location_Type'].value_counts()

2    3350
1    2785
0    2388
Name: Outlet_Location_Type, dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   float64
 9   Outlet_Location_Type       8523 non-null   int64  
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(5), int64(2), object(5)
memory usage: 799.2+ KB


In [ ]:
df['Outlet_Type'].value_counts()

Supermarket Type1    5577
Grocery Store        1083
Supermarket Type3     935
Supermarket Type2     928
Name: Outlet_Type, dtype: int64

In [ ]:
df.drop(columns = 'Item_Identifier', inplace = True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Weight                7060 non-null   float64
 1   Item_Fat_Content           8523 non-null   object 
 2   Item_Visibility            8523 non-null   float64
 3   Item_Type                  8523 non-null   object 
 4   Item_MRP                   8523 non-null   float64
 5   Outlet_Identifier          8523 non-null   object 
 6   Outlet_Establishment_Year  8523 non-null   int64  
 7   Outlet_Size                6113 non-null   float64
 8   Outlet_Location_Type       8523 non-null   int64  
 9   Outlet_Type                8523 non-null   object 
 10  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(5), int64(2), object(4)
memory usage: 732.6+ KB


In [ ]:
df['Outlet_Type'].value_counts()

Supermarket Type1    5577
Grocery Store        1083
Supermarket Type3     935
Supermarket Type2     928
Name: Outlet_Type, dtype: int64

In [ ]:
X = df.drop(columns = 'Item_Outlet_Sales')
y = df['Item_Outlet_Sales']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
scaler = StandardScaler()
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

mean_imputer = SimpleImputer(strategy = 'mean')
constant_imputer = SimpleImputer(strategy='constant', fill_value='missing')

In [ ]:
num_pipe=make_pipeline(mean_imputer, scaler)
cat_pipe = make_pipeline(constant_imputer, ohe)

In [ ]:
cat_cols = make_column_selector(dtype_include = 'object')
num_cols = make_column_selector(dtype_include='number')

num_tuple = (num_pipe, num_cols)
cat_tuple = (cat_pipe, cat_cols)

In [ ]:
preprocessor = make_column_transformer(num_tuple, cat_tuple, remainder='drop')

In [ ]:
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fd618ccdfa0>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fd618ccda00>)])

In [ ]:
preprocessor.transform(X_train)
preprocessor.transform(X_test)

array([[ 0.33100885, -0.77664625, -0.99881554, ...,  1.        ,
         0.        ,  0.        ],
       [-1.17989246,  0.1003166 , -1.58519423, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.37844688, -0.48299432, -1.59578435, ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [-1.13957013,  1.21832428,  1.09397975, ...,  1.        ,
         0.        ,  0.        ],
       [-1.49772727, -0.77809567, -0.36679966, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.52076098, -0.77976293,  0.11221189, ...,  1.        ,
         0.        ,  0.        ]])

In [ ]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()

In [ ]:
from sklearn.pipeline import make_pipeline
reg_pipe = make_pipeline(preprocessor, reg)

In [ ]:
reg_pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fd618ccdfa0>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fd618ccda00>)])),
                ('linearregression', LinearRegression())])

In [ ]:
predictions = reg_pipe.predict(X_test)

In [ ]:
prediction_df = X_test.copy()
prediction_df['True Median Price'] = y_test
prediction_df['Predicted Median Price']= predictions
prediction_df['Error'] = predictions - y_test
prediction_df.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,True Median Price,Predicted Median Price,Error
7503,14.300,Low Fat,0.026300,Frozen Foods,79.4302,OUT013,1987,2.0,2,Supermarket Type1,1743.0644,1360.421875,-382.642525
2957,7.930,Low Fat,0.071136,Health and Hygiene,42.7086,OUT046,1997,0.0,0,Supermarket Type1,356.8688,717.546875,360.678075
7031,14.500,Regular,0.041313,Canned,42.0454,OUT049,1999,1.0,0,Supermarket Type1,377.5086,881.625000,504.116400
1084,NaN,Regular,0.044767,Soft Drinks,173.7054,OUT027,1985,1.0,2,Supermarket Type3,5778.4782,4235.296875,-1543.181325
856,10.195,Regular,0.012456,Meat,197.5110,OUT035,2004,0.0,1,Supermarket Type1,2356.9320,3342.375000,985.443000


In [ ]:
train_pred = reg_pipe.predict(X_train)
test_pred = reg_pipe.predict(X_test)

In [ ]:
train_r2 = r2_score(y_train, train_pred)
test_r2 = r2_score(y_test, test_pred)

print(f'Model training R2: {train_r2}')
print(f'Model testing R2: {test_r2}')

Model training R2: 0.5615550228827033
Model testing R2: 0.5671033343344658


In [ ]:
train_MSE = mean_squared_error(y_train, train_pred)
test_MSE = mean_squared_error(y_test, test_pred)

train_RMSE = np.sqrt(train_MSE)
test_RMSE = np.sqrt(test_MSE)

print(f'Model Training RMSE: {train_RMSE}')
print(f'Model Testing RMSE: {test_RMSE}')

Model Training RMSE: 1139.1042483683398
Model Testing RMSE: 1092.8641583190656


In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
dec_tree = DecisionTreeRegressor(random_state=42)

In [ ]:
dec_pipeline = make_pipeline(preprocessor, dec_tree)

In [ ]:
dec_pipeline.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fd618ccdfa0>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fd618ccda00>)])),
                ('decisiontreeregressor',
                 DecisionTreeRegressor(random_state=42))])

In [ ]:
dec_train_preds = dec_pipeline.predict(X_train)
dec_test_preds = dec_pipeline.predict(X_test)

In [ ]:
train_score = dec_pipeline.score(X_train, y_train)
test_score = dec_pipeline.score(X_test, y_test)
print(train_score)
print(test_score)

1.0
0.1805654820183923


In [ ]:
dec_train_MSE = mean_squared_error(y_train, dec_train_preds)
dec_test_MSE = mean_squared_error(y_test, dec_test_preds)

dec_train_RMSE = np.sqrt(dec_train_MSE)
dec_test_RMSE = np.sqrt(dec_test_MSE)

print(f'Model Training RMSE: {dec_train_RMSE}')
print(f'Model Testing RMSE: {dec_test_RMSE}')

Model Training RMSE: 4.925864104892086e-15
Model Testing RMSE: 1503.596097346641


In [ ]:
#Tune the model
#List of values to try for max_depth
depths = list(range(2, 36)) #Will try every value between 2 and 36
#Data frame to store the score for each value of max_depth
scores = pd.DataFrame(index=depths, columns = ['Test Score', 'Train Score'])

for depth in depths:
  dec_tree = DecisionTreeRegressor(max_depth = depth, random_state=42)
  dec_pipeline = make_pipeline(preprocessor, dec_tree)
  dec_pipeline.fit(X_train, y_train)
  train_score = dec_pipeline.score(X_train, y_train)
  test_score = dec_pipeline.score(X_test, y_test)
  scores.loc[depth, 'Train Score' ]=train_score
  scores.loc[depth, 'Test Score']=test_score



In [ ]:
scores.sort_values(by='Test Score', ascending=False)

,Test Score,Train Score
5,0.59471,0.60394
4,0.584005,0.582625
6,0.582274,0.615161
7,0.576984,0.626843
8,0.560225,0.643832
9,0.536316,0.665793
10,0.527951,0.685718
3,0.524222,0.524218
11,0.503515,0.710195
12,0.473303,0.73582


In [ ]:

depths = list(range(2, 36))
scores = pd.DataFrame(index=depths, columns = ['Test Score', 'Train Score'])

for depth in depths:
  dec_tree = DecisionTreeRegressor(max_depth = depth, random_state=42)
  dec_pipeline = make_pipeline(preprocessor, dec_tree)
  dec_pipeline.fit(X_train, y_train)
  train_tuned_pred = dec_pipeline.predict(X_train)
  test_tuned_pred = dec_pipeline.predict(X_test)
  trainRMSE_tuned = np.sqrt(mean_squared_error(y_train, train_tuned_pred))
  testRMSE_tuned= np.sqrt(mean_squared_error(y_test, test_tuned_pred))
  scores.loc[depth, 'Train Score' ]=trainRMSE_tuned
  scores.loc[depth, 'Test Score']=testRMSE_tuned

In [ ]:
scores.sort_values(by='Test Score', ascending=True)

,Test Score,Train Score
5,1057.44313,1082.64619
4,1071.317476,1111.396239
6,1073.54409,1067.198879
7,1080.319633,1050.877167
8,1101.512357,1026.675415
9,1131.058462,994.520074
10,1141.214886,964.41953
3,1145.714222,1186.614963
11,1170.380261,926.102834
12,1205.464813,884.211493


1092(linear regression) vs 1057(dec tree) - dec tree is better.

In [ ]:
tuned_dectree = DecisionTreeRegressor(max_depth=5, random_state=42)
tuned_decpipe=make_pipeline(preprocessor, tuned_dectree)
tuned_decpipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fd618ccdfa0>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fd618ccda00>)])),
                ('decisiontreeregressor',
                 DecisionTreeRegressor(max_depth=5, random_state=42))])

Evalaute both models on all metrics

In [ ]:
## Create a function to take the true and predicted values
## and print MAE, MSE, RMSE, and R2 metrics for a model
def model_metrics(pipe, x_train, y_train, x_test, y_test, 
                       model_name='Regression Model', ):
  ## Train
  mae = mean_absolute_error(y_train, pipe.predict(x_train))
  mse = mean_squared_error(y_train, pipe.predict(x_train))
  rmse = np.sqrt(mean_squared_error(y_train, pipe.predict(x_train)))
  r2 = r2_score(y_train, pipe.predict(x_train))
  print(f'{model_name} Train Scores')
  print(f'MAE: {mae:,.4f} \nMSE: {mse:,.4f} \nRMSE: {rmse:,.4f} \nR2: {r2:.4f}\n')

  ## Test
  mae = mean_absolute_error(y_test, pipe.predict(x_test))
  mse = mean_squared_error(y_test, pipe.predict(x_test))
  rmse = np.sqrt(mean_squared_error(y_test, pipe.predict(x_test)))
  r2 = r2_score(y_test, pipe.predict(x_test))

  # Display the metrics for the model
  print(f'{model_name} Test Scores')
  print(f'MAE: {mae:,.4f} \nMSE: {mse:,.4f} \nRMSE: {rmse:,.4f} \nR2: {r2:.4f}\n')

In [ ]:
model_metrics(reg_pipe, X_train, y_train, X_test, y_test, model_name='linreg')

linreg Train Scores
MAE: 847.1319 
MSE: 1,297,558.4887 
RMSE: 1,139.1042 
R2: 0.5616

linreg Test Scores
MAE: 804.1214 
MSE: 1,194,352.0685 
RMSE: 1,092.8642 
R2: 0.5671



In [ ]:
model_metrics(tuned_decpipe, X_train, y_train, X_test, y_test, model_name='decscores')

decscores Train Scores
MAE: 762.6102 
MSE: 1,172,122.7729 
RMSE: 1,082.6462 
R2: 0.6039

decscores Test Scores
MAE: 738.3173 
MSE: 1,118,185.9731 
RMSE: 1,057.4431 
R2: 0.5947

